# RecommendationEvaluator Execution in Jupyter Notebook

This notebook demonstrates how to use the `RecommendationEvaluator` class to evaluate recommendations from processed data files. The following steps outline the process of setting up and executing the evaluation.

## Step 1: Import Necessary Libraries

First, import all necessary libraries and modules.

In [ ]:
import json
import logging
import os
import re
import pandas as pd
import html
from tqdm import tqdm
import string
from utils.metrics4rec import evaluate_all
import openpyxl  # Ensure this is installed for Excel handling


## Step 2: Define the RecommendationEvaluator Class

The `RecommendationEvaluator` class encapsulates all the functionality required to evaluate recommendations. Ensure the class definition is included in the notebook.

In [ ]:
class RecommendationEvaluator:
    def __init__(self, processed_data_path, min_recommendations=1):
        """
        Initialize the RecommendationEvaluator with the path to processed data and minimum recommendations.
        """
        logging.basicConfig(level=logging.INFO)
        self.processed_data_path = processed_data_path
        self.min_recommendations = min_recommendations
        self.processed_data = self.load_json_data(processed_data_path)
        self.failed_user_ids = []  # List to store user IDs with JSONDecodeError

    @staticmethod
    def load_json_data(file_path):
        """
        Load and return data from a JSON file.
        """
        with open(file_path, 'r') as file:
            return json.load(file)
        
    def jaccard_similarity(self, s1, s2):
        """
        Calculate Jaccard similarity between two strings.
        """
        set1 = set(s1.split())
        set2 = set(s2.split())

        intersection = set1.intersection(set2)
        union = set1.union(set2)

        if not union:
            return 100.0  # Avoid division by zero

        similarity = len(intersection) / len(union) * 100
        return similarity

    def find_most_similar_jaccard(self, target, recommendations, strict=False):
        """
        Find the most similar recommendation to the target using Jaccard similarity.
        """
        max_id = 0
        max_sim = 0
        for id, s2 in enumerate(recommendations):
            sim = self.jaccard_similarity(target, s2)
            if sim > max_sim:
                max_sim = sim
                max_id = id

        if strict:
            target_first_word = target.split()[0] if target.split() else ""
            recommendation_first_word = recommendations[max_id].split()[0] if recommendations[max_id].split() else ""
            if target_first_word == recommendation_first_word:
                return max_id, max_sim
            else:
                return None, 0

        return max_id, max_sim
    
    def prepare_evaluation_data(self):
        """
        Prepare prediction and ground truth data for evaluation.
        """
        predictions, ground_truth = {}, {}

        json_failed_count = 0
        self.failed_user_ids = []

        for user_id in tqdm(self.processed_data, desc="Evaluating"):
            info = self.processed_data[user_id]
            ground_truth[user_id] = set([html.unescape(info['target']['titles'][0])])

            if "api_response" in info and isinstance(info['api_response'], str):
                try:
                    info['api_response'] = json.loads(info['api_response'])
                except json.JSONDecodeError:
                    json_failed_count += 1
                    self.failed_user_ids.append(user_id)

            if "api_response" in info and isinstance(info['api_response'], dict):
                recommendations = info['api_response'].get('recommendations', [])
            else:
                recommendations = []

            if len(recommendations) == 0:
                continue
            if recommendations:
                recommendations = [html.unescape(_) for _ in recommendations]
                max_id, _ = self.find_most_similar_jaccard(target=info['target']['titles'][0], recommendations=recommendations, strict=True)
                mapped_titles = [_ for _ in recommendations]
                if max_id is not None:
                    mapped_titles[max_id] = list(ground_truth[user_id])[0]

                predictions[user_id] = {title: len(mapped_titles) - idx for idx, title in enumerate(mapped_titles)}
            else:
                predictions[user_id] = {f"Item_{i}": self.min_recommendations - i for i in range(self.min_recommendations)}
        print("failed parsed json:{}_{}".format(json_failed_count, len(self.failed_user_ids)))
        return predictions, ground_truth

    def evaluate(self):
        """
        Evaluate recommendations and return metrics for different topks.
        """
        predictions, ground_truth = self.prepare_evaluation_data()
        metrics = {f'@{k}': evaluate_all(predictions, ground_truth, topk=k) for k in [1, 3, 5, 10, 20, 30]}
        return metrics


## Step 3: Define the Helper Function to Find Evaluation Paths

Create a function to traverse the directory and find all paths to `processed_data.json` files within directories whose names contain specified substrings.


In [ ]:
def find_evaluation_paths(root_path, containing):
    """
    Traverse the directory to find all paths to 'processed_data.json' files
    within directories whose names contain specified substrings.
    """
    for dirpath, dirnames, filenames in os.walk(root_path):
        for filename in filenames:
            if filename == 'processed_data.json' or filename == 'result.json' and any(contained in dirpath for contained in containing):
                yield os.path.join(dirpath, filename)



## Step 4: Define the Main Function to Execute the Evaluation Process

Create the `main` function to execute the evaluation process for multiple datasets. This function will handle loading the data, initializing the evaluator, and saving the results.



In [ ]:
def main(root_path, containing):
    """
    Main function to execute the evaluation process for multiple datasets.
    """
    # Settings and paths
    dataset_name_list = ['beauty', 'clothing', 'toys', 'sports']
    for dataset_name in dataset_name_list:
        max_seq_len = 10
        formatted_root_path = root_path.format(dataset_name, max_seq_len)
        # Initialize variables
        results = []

        # Logging to debug path discovery
        logging.basicConfig(level=logging.INFO)

        for file_path_data in find_evaluation_paths(formatted_root_path, containing):
            # Log the found paths for debugging
            logging.info(f"Found processed_data.json at: {file_path_data}")

            # Parse template_id and model_name from the directory name
            dir_name_components = os.path.basename(os.path.dirname(file_path_data)).split('_')
            template_id = dir_name_components[1]
            model_name = '_'.join(dir_name_components[2:])

            # Create evaluator and evaluate
            evaluator = RecommendationEvaluator(file_path_data)            
            evaluation_metrics = evaluator.evaluate()

            # Prepare results for all topks
            logging.info(evaluation_metrics)  # Log the evaluation metrics for debugging
            result = {
                "dataset_name": dataset_name,
                "model_name": model_name,
                "max_seq_len": max_seq_len,
                "template_id": template_id,
                "cnt": evaluation_metrics[f'@1'][1]['cnt']
            }

            # Extract NDCG and Hits for each k and update the result dictionary
            for k in [1, 3, 5, 10, 20, 30]:
                result[f"NDCG@{k}"] = evaluation_metrics[f'@{k}'][1]['ndcg']
            for k in [1, 3, 5, 10, 20, 30]:
                result[f"Hits@{k}"] = evaluation_metrics[f'@{k}'][1]['hit']

            results.append(result)

        # Convert results to a DataFrame and save as Excel
        results_df = pd.DataFrame(results)
        results_df.to_excel("{}_{}_results.xlsx".format(dataset_name, max_seq_len), index=False)

if __name__ == "__main__":
    # Define the root path and the containing substrings
    root_path = input("Enter the root path (use placeholders {} for dataset and sequence length): ")
    containing = input("Enter the substrings to search for (comma-separated): ").split(',')

    # Run the main function with the defined parameters
    main(root_path, containing)